In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
# configs de performance

spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)

In [0]:
# paths

path_silver = "abfss://silver@storagedatanexus.dfs.core.windows.net/"
target_table = "gold.rst_exp_imp_pais"

In [0]:
df_export = spark.read.table("silver_comercio_ext_estatisticas.tb_exportacoes") \
    .select(
        "CO_ANO", 
        "CO_MES", 
        "CO_PAIS",
        "CO_NCM", 
        "VL_FOB", 
    )

df_import = spark.read.table("silver_comercio_ext_estatisticas.tb_importacoes") \
    .select(
        "CO_ANO", 
        "CO_MES", 
        "CO_PAIS",
        "CO_NCM", 
        "VL_FOB", 
    )
    
df_dim_pais = spark.read.table("silver_comercio_ext_auxiliares.tb_auxiliar_paises") \
    .select(
        "CO_PAIS",
    )


In [0]:
df_dim_isic_ncm = spark.read.table("silver_comercio_ext_indices.tb_nomenclatura_mercosul") \
    .select(
        "CO_NCM", 
        "CO_ISIC_CLASSE",
    )

df_dim_isic = spark.read.table("silver_comercio_ext_indices.tb_referencia_ncm_isic") \
    .select(
        "CO_ISIC_CLASSE",
        "CO_ISIC_DIVISAO"
    )


In [0]:
from pyspark.sql import functions as F

# =========================================================
# Agregação exportações
# =========================================================
exp_mensal = (
    df_export
    .groupBy("CO_ANO", "CO_MES", "CO_PAIS", "CO_NCM")
    .agg(F.sum("VL_FOB").alias("total_exportado"))
)

# =========================================================
# Agregação importações
# =========================================================
imp_mensal = (
    df_import
    .groupBy("CO_ANO", "CO_MES", "CO_PAIS", "CO_NCM")
    .agg(F.sum("VL_FOB").alias("total_importado"))
)

# =========================================================
# Union export + import (FULL)
# =========================================================
fato_mensal = (
    exp_mensal.alias("e")
    .join(
        imp_mensal.alias("i"),
        ["CO_ANO", "CO_MES", "CO_PAIS", "CO_NCM"],
        "full"
    )
    .select(
        F.coalesce(F.col("e.CO_ANO"),  F.col("i.CO_ANO")).cast("int").alias("ano_operacao"),
        F.coalesce(F.col("e.CO_MES"),  F.col("i.CO_MES")).cast("int").alias("mes_operacao"),
        F.coalesce(F.col("e.CO_PAIS"), F.col("i.CO_PAIS")).alias("CO_PAIS"),
        F.coalesce(F.col("e.CO_NCM"),  F.col("i.CO_NCM")).alias("CO_NCM"),
        F.coalesce(F.col("e.total_exportado"), F.lit(0)).alias("total_exportado"),
        F.coalesce(F.col("i.total_importado"), F.lit(0)).alias("total_importado"),
    )
)

# =========================================================
# Dimensões
# =========================================================
dim_pais = df_dim_pais.dropDuplicates(["CO_PAIS"])
dim_isic_ncm = df_dim_isic_ncm.dropDuplicates(["CO_NCM"])
dim_isic = df_dim_isic.dropDuplicates(["CO_ISIC_CLASSE"])

# =========================================================
# Fato final enriquecido
# =========================================================
rst_exp_imp_pais = (
    fato_mensal
    .join(dim_pais, on="CO_PAIS", how="left")
    .join(dim_isic_ncm, on="CO_NCM", how="left")
    .join(dim_isic, on="CO_ISIC_CLASSE", how="left")
    .withColumn(
        "ano_mes",
        F.concat_ws(
            "-",
            F.col("ano_operacao"),
            F.lpad(F.col("mes_operacao").cast("string"), 2, "0")
        )
    )
    .withColumn(
        "saldo",
        (
            F.col("total_exportado").cast("float")
            - F.col("total_importado").cast("float")
        )
    )
    .select(
        "ano_operacao",
        "mes_operacao",
        "ano_mes",

        F.col("CO_PAIS").cast("string").alias("cod_pais"),
        F.col("CO_NCM").cast("string").alias("cod_ncm"),
        F.col("CO_ISIC_DIVISAO").cast("string").alias("cod_div_isic"),
        F.col("CO_ISIC_CLASSE").cast("string").alias("cod_isic"),

        F.col("total_exportado").cast("float"),
        F.col("total_importado").cast("float"),
        F.col("saldo").cast("float")
    )
    .orderBy(
        "ano_operacao",
        "mes_operacao",
        "cod_pais",
        "cod_isic",
        "cod_ncm"
    )
)


In [0]:
display(rst_exp_imp_pais.limit(10))

In [0]:
rst_exp_imp_pais.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ano_operacao", "mes_operacao") \
    .saveAsTable(target_table)

spark.sql(f"OPTIMIZE gold.rst_exp_imp_pais ZORDER BY (cod_pais, cod_isic)")
